In [156]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyairports.airports import Airports
from datetime import datetime

# Importing Dataset
data=pd.read_csv('Data/Flight_Data_New.csv',usecols=['searchDate', 'flightDate', 'startingAirport',
       'starting_airport_name', 'destinationAirport',
       'destination_airport_name', 
       'travelDuration', 'isBasicEconomy', 'isRefundable', 'isNonStop',
       'seatsRemaining', 'totalTravelDistance',
       'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode',
       'segmentsAirlineName',
       'segmentsEquipmentDescription','segmentsCabinCode','totalFare'],nrows=1000)
data.head()

,searchDate,flightDate,startingAirport,starting_airport_name,destinationAirport,destination_airport_name,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsEquipmentDescription,segmentsCabinCode
0,2022-05-29,2022-06-18,LAX,Los Angeles Intl,ATL,Hartsfield Jackson Atlanta Intl,PT4H19M,False,False,True,1028.61,7,1943.0,ATL,LAX,Delta,Airbus A321,coach
1,2022-05-03,2022-06-24,CLT,Charlotte Douglas Intl,LAX,Los Angeles Intl,PT8H10M,False,False,False,537.10,9,2171.0,ATL||LAX,CLT||ATL,Delta||Delta,Boeing 717||Airbus A321,coach||coach
2,2022-05-02,2022-06-30,LGA,La Guardia,ATL,Hartsfield Jackson Atlanta Intl,PT11H54M,False,False,False,382.21,9,1700.0,MIA||ATL,LGA||MIA,Delta||Delta,Boeing 737-800||Boeing 737-900,coach||coach
3,2022-06-10,2022-08-04,LAX,Los Angeles Intl,SFO,San Francisco Intl,PT1H20M,False,False,True,218.60,7,339.0,SFO,LAX,Alaska Airlines,Embraer 175,coach
4,2022-06-24,2022-07-20,LGA,La Guardia,DFW,Dallas Fort Worth Intl,PT6H41M,True,False,False,207.60,9,1487.0,ATL||DFW,LGA||ATL,Delta||Delta,Airbus A320||Airbus A321,coach||coach


In [157]:
# Extracting Number of days between flight Date and search date
num_days=[]
for dt in range(data.shape[0]):
    num_day=datetime.strptime(data['flightDate'][dt], '%Y-%m-%d').date()-datetime.strptime(data['searchDate'][dt], '%Y-%m-%d').date()
    num_day=num_day.days
    num_days.append(num_day)
data['NumDays']=num_days

column_index = data.columns.get_loc('flightDate') + 1
data.insert(column_index, 'NumDays', data.pop('NumDays'))

data.head()

,searchDate,flightDate,NumDays,startingAirport,starting_airport_name,destinationAirport,destination_airport_name,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsEquipmentDescription,segmentsCabinCode
0,2022-05-29,2022-06-18,20,LAX,Los Angeles Intl,ATL,Hartsfield Jackson Atlanta Intl,PT4H19M,False,False,True,1028.61,7,1943.0,ATL,LAX,Delta,Airbus A321,coach
1,2022-05-03,2022-06-24,52,CLT,Charlotte Douglas Intl,LAX,Los Angeles Intl,PT8H10M,False,False,False,537.10,9,2171.0,ATL||LAX,CLT||ATL,Delta||Delta,Boeing 717||Airbus A321,coach||coach
2,2022-05-02,2022-06-30,59,LGA,La Guardia,ATL,Hartsfield Jackson Atlanta Intl,PT11H54M,False,False,False,382.21,9,1700.0,MIA||ATL,LGA||MIA,Delta||Delta,Boeing 737-800||Boeing 737-900,coach||coach
3,2022-06-10,2022-08-04,55,LAX,Los Angeles Intl,SFO,San Francisco Intl,PT1H20M,False,False,True,218.60,7,339.0,SFO,LAX,Alaska Airlines,Embraer 175,coach
4,2022-06-24,2022-07-20,26,LGA,La Guardia,DFW,Dallas Fort Worth Intl,PT6H41M,True,False,False,207.60,9,1487.0,ATL||DFW,LGA||ATL,Delta||Delta,Airbus A320||Airbus A321,coach||coach


In [158]:
# Flight Day (M,T,W,etc)

data['flightDate'] = pd.to_datetime(data['flightDate'])
data['Day_of_Flight'] = data['flightDate'].dt.day_name()  


column_index = data.columns.get_loc('flightDate') + 1
data.insert(column_index, 'Day_of_Flight', data.pop('Day_of_Flight'))

days={'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6, 'Sunday':7}
data['Day_of_Flight']=data['Day_of_Flight'].map(days)

In [159]:
#Number of Hours
def calculate_total_minutes(duration):
    hour=0
    min=0
    try: 
        hour=int(duration.split('H')[0][-1])
        try:
            min= int(duration.split('H')[1][:-1])
        except:
            min=0
    except:
        try:
            min= int(duration.split('H')[1][:-1])
        except:
            min=0
    return (hour*60)+min

data['Flight_time_in_minutes'] = data['travelDuration'].apply(calculate_total_minutes)

column_index = data.columns.get_loc('travelDuration') + 1
data.insert(column_index, 'Flight_time_in_minutes', data.pop('Flight_time_in_minutes'))

data.drop(['travelDuration'],axis=1,inplace=True) 

In [160]:
data.head()

,searchDate,flightDate,Day_of_Flight,NumDays,startingAirport,starting_airport_name,destinationAirport,destination_airport_name,Flight_time_in_minutes,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsEquipmentDescription,segmentsCabinCode
0,2022-05-29,2022-06-18,6,20,LAX,Los Angeles Intl,ATL,Hartsfield Jackson Atlanta Intl,259,False,False,True,1028.61,7,1943.0,ATL,LAX,Delta,Airbus A321,coach
1,2022-05-03,2022-06-24,5,52,CLT,Charlotte Douglas Intl,LAX,Los Angeles Intl,490,False,False,False,537.10,9,2171.0,ATL||LAX,CLT||ATL,Delta||Delta,Boeing 717||Airbus A321,coach||coach
2,2022-05-02,2022-06-30,4,59,LGA,La Guardia,ATL,Hartsfield Jackson Atlanta Intl,114,False,False,False,382.21,9,1700.0,MIA||ATL,LGA||MIA,Delta||Delta,Boeing 737-800||Boeing 737-900,coach||coach
3,2022-06-10,2022-08-04,4,55,LAX,Los Angeles Intl,SFO,San Francisco Intl,80,False,False,True,218.60,7,339.0,SFO,LAX,Alaska Airlines,Embraer 175,coach
4,2022-06-24,2022-07-20,3,26,LGA,La Guardia,DFW,Dallas Fort Worth Intl,401,True,False,False,207.60,9,1487.0,ATL||DFW,LGA||ATL,Delta||Delta,Airbus A320||Airbus A321,coach||coach


In [161]:
#Boolean variables
data[[ 'isBasicEconomy', 'isRefundable', 'isNonStop']].head()

t_f={True:1,False:0}
data['isBasicEconomy']=data['isBasicEconomy'].map(t_f)
data['isRefundable']=data['isRefundable'].map(t_f)
data['isNonStop']=data['isNonStop'].map(t_f)

data[[ 'isBasicEconomy', 'isRefundable', 'isNonStop']].head()

,isBasicEconomy,isRefundable,isNonStop
0,0,0,1
1,0,0,0
2,0,0,0
3,0,0,1
4,1,0,0


In [162]:
data.head()

,searchDate,flightDate,Day_of_Flight,NumDays,startingAirport,starting_airport_name,destinationAirport,destination_airport_name,Flight_time_in_minutes,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsEquipmentDescription,segmentsCabinCode
0,2022-05-29,2022-06-18,6,20,LAX,Los Angeles Intl,ATL,Hartsfield Jackson Atlanta Intl,259,0,0,1,1028.61,7,1943.0,ATL,LAX,Delta,Airbus A321,coach
1,2022-05-03,2022-06-24,5,52,CLT,Charlotte Douglas Intl,LAX,Los Angeles Intl,490,0,0,0,537.10,9,2171.0,ATL||LAX,CLT||ATL,Delta||Delta,Boeing 717||Airbus A321,coach||coach
2,2022-05-02,2022-06-30,4,59,LGA,La Guardia,ATL,Hartsfield Jackson Atlanta Intl,114,0,0,0,382.21,9,1700.0,MIA||ATL,LGA||MIA,Delta||Delta,Boeing 737-800||Boeing 737-900,coach||coach
3,2022-06-10,2022-08-04,4,55,LAX,Los Angeles Intl,SFO,San Francisco Intl,80,0,0,1,218.60,7,339.0,SFO,LAX,Alaska Airlines,Embraer 175,coach
4,2022-06-24,2022-07-20,3,26,LGA,La Guardia,DFW,Dallas Fort Worth Intl,401,1,0,0,207.60,9,1487.0,ATL||DFW,LGA||ATL,Delta||Delta,Airbus A320||Airbus A321,coach||coach


In [163]:
#number of segments
num_segments=[]
for flight in range(data.shape[0]):
    num_segments.append(data['segmentsArrivalAirportCode'][flight].count('|')+1)


data['Num_Segments']=num_segments
column_index = data.columns.get_loc('segmentsArrivalAirportCode') + 1
data.insert(column_index, 'Num_Segments', data.pop('Num_Segments'))


In [164]:
#cabin code weight

cabin_code_weights={'coach':1,'premium coach':2,'business':3,'first':4}
weighted_cabin_code=[]

for cabin_code in range(data.shape[0]):
    weight=0
    segment_c_code=data['segmentsCabinCode'][cabin_code].split("||")
    for c_code in segment_c_code:
        weight += cabin_code_weights[c_code]
    weighted_cabin_code.append(weight)

data['cabin_code_weight']=weighted_cabin_code
column_index = data.columns.get_loc('segmentsCabinCode') + 1
data.insert(column_index, 'cabin_code_weight', data.pop('cabin_code_weight'))

In [165]:
distinct_airline_names=[]
unique_cabin_codes_combo=list(data['segmentsAirlineName'].unique())
for c_code in range(len(unique_cabin_codes_combo)):
    segment_c_code=unique_cabin_codes_combo[c_code].split("||")
    for segment in segment_c_code:
        if segment not in distinct_airline_names:
            distinct_airline_names.append(segment)
        else:
            continue
print(distinct_airline_names)

['Delta', 'Alaska Airlines', 'United', 'American Airlines', 'Spirit Airlines', 'Southern Airways Express', 'JetBlue Airways', 'Cape Air', 'Frontier Airlines', 'Sun Country Airlines', 'Key Lime Air']


In [169]:
distinct_airplane_names=[]
unique_airplane_names_combo=list(data['segmentsEquipmentDescription'].unique())
for c_code in range(len(unique_airplane_names_combo)):
    try:
        segment_c_code=unique_airplane_names_combo[c_code].split("||")
        for segment in segment_c_code:
            if segment not in distinct_airplane_names:
                distinct_airplane_names.append(segment)
            else:
                continue
    except:
        print(unique_airplane_names_combo[c_code])


nan


In [170]:
data['segmentsEquipmentDescription'].isna().sum()

26